# **MSc Neural Network Experiments**

In [ ]:
import json
import numpy as np
import sys
sys.path.append("../utilities")
import U_config as cfg
import os

# Data Loader
import data_loader

# Tensorflow Libraries
import tensorflow as tf
import visualkeras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, UpSampling2D, concatenate, MaxPooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

# MIT Evidential-Deep Learning Library
import evidential_deep_learning as edl

### **Neural Network**

Two networks can be created - the traditional U-Net structure (with a modified regressive output), and the 'evidential' network that employs the MIT Evidential Deep Learnning library. The latter is used to generate spatial plots of uncertainty within this study.

In [ ]:
# Implements a modified U-Network, with one less decoder/ encoder block to accomodate the uneven image dimensions that arise when downsampling.
def create_model(TOTAL_CHANNELS, evidential):
    input = Input((104, 104, TOTAL_CHANNELS))
    
    c1 = Conv2D(64, 3, activation="relu", padding='same')(input)
    c1 = Conv2D(64, 3, activation="relu", padding='same')(c1)
    bn1 = BatchNormalization(axis=-1)(c1)
    pool1 = MaxPooling2D(pool_size=(2,2))(bn1)

    c2 = Conv2D(128, 3, activation="relu", padding='same')(pool1)
    c2 = Conv2D(128, 3, activation="relu", padding='same')(c2)
    bn2 = BatchNormalization(axis=-1)(c2)
    pool2 = MaxPooling2D(pool_size=(2,2))(bn2)

    c3 = Conv2D(256, 3, activation="relu", padding='same')(pool2)
    c3 = Conv2D(256, 3, activation="relu", padding='same')(c3)
    bn3 = BatchNormalization(axis=-1)(c3)
    pool3 = MaxPooling2D(pool_size=(2,2))(bn3)

    c4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    c4 = Conv2D(512, 3, activation='relu', padding='same')(c4)
    bn5 = BatchNormalization(axis=-1)(c4)

    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2,2), interpolation='nearest')(bn5))
    m7 = concatenate([bn3,up7], axis=3)
    c7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(m7)
    c7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(c7)
    bn7 = BatchNormalization(axis=-1)(c7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2), interpolation='nearest')(bn7))
    m8 = concatenate([bn2,up8], axis=3)
    c8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(m8)
    c8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(c8)
    bn8 = BatchNormalization(axis=-1)(c8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2), interpolation='nearest')(bn8))
    m9 = concatenate([c1,up9], axis=3)
    c9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(m9)
    c9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(c9)
    c9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(c9)

    if evidential:
        fin = edl.layers.Conv2DNormalGamma(kernel_size=1, filters=1)(c9)
        model = Model(input, fin)
        model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=edl.losses.EvidentialRegression)
    else:
        fin = Conv2D(1,1,activation='sigmoid')(c9)
        model = Model(input, fin)
        model.compile(optimizer=tf.keras.optimizers.Adam(), loss="mean_squared_error", metrics="accuracy")

    return model

### **Data Setup**

In [ ]:
# Loads a data-loader config & produces a Data Loader item for it.
def load_data_and_channels(config_file):
    cfg = open(config_file)
    cfg=json.load(cfg)
    data_object = data_loader.ThesisDataLoader(cfg)
    channel_count = sum(list(cfg['variables'].values()))
    return data_object, channel_count

## **Transfer Learning Setup**

Pre-Trains the network on the preliminary 1980-2010 dataset (using evidential or traditional model)

In [2]:
def pre_train(EXPERIMENT_ID, experiment_path, destination, evidential, EPOCHS):
    """
        Acquires the model and data loader.
        Performs pre training.
        Saves incremental improvements of the model.
    """
    if (evidential):
        print("Training an Evidential Model.")

    # Get data loader
    data, channels = load_data_and_channels(f"./configs/{EXPERIMENT_ID}/pre_training.json")
    
    # Get model:
    baseline_model = create_model(channels, evidential)
    
    save_path = os.path.join(experiment_path, f'pre_training_{EXPERIMENT_ID}.h5')

    # Setup model callbacks (early stopping & save checkpoints)
    es = EarlyStopping(monitor='loss', verbose=1, patience=5)
    mc = ModelCheckpoint(destination, monitor='loss', mode='min', verbose=1, save_best_only=True)

    # Pre-train the model:
    baseline_model.fit(data, epochs=EPOCHS, verbose=1, callbacks = [es,mc])

### Copying Weights (Transfer Learning)
This block is taken from 'Digital Sreeni':
YouTube: "Tips Tricks 20 - Understanding transfer learning for different size and channel inputs"
https://github.com/bnsreenu/python_for_microscopists/blob/master/Tips_tricks_20_Understanding%20transfer%20learning%20for%20different%20size%20and%20channel%20inputs.py


In [ ]:
# CODE from this block is taken from 'Digital Sreeni':
# YouTube: "Tips Tricks 20 - Understanding transfer learning for different size and channel inputs"
# https://github.com/bnsreenu/python_for_microscopists/blob/master/Tips_tricks_20_Understanding%20transfer%20learning%20for%20different%20size%20and%20channel%20inputs.py
def copy_weights(weights, num_channels_to_fill):  
  average_weights = np.mean(weights, axis=-2).reshape(weights[:,:,-1:,:].shape)  
  wts_copied_to_mult_channels = np.tile(average_weights, (num_channels_to_fill, 1)) 
  return(wts_copied_to_mult_channels)

def create_transfer_network(source, evidential=False):
    data, new_channels = load_data_and_channels(f"./configs/{EXPERIMENT_ID}/post_training.json")
    old_channel_count = sum(list(json.load(open(f"./configs/{EXPERIMENT_ID}/pre_training.json"))['variables'].values()))

    print(f'\nThe old model had {old_channel_count} channels, the new model has {new_channels}.\n')

    # Get setup:

    if evidential:
        pre_trained_model = tf.keras.models.load_model(source, custom_objects={'Conv2DNormalGamma': edl.layers.Conv2DNormalGamma, 'EvidentialRegression':edl.losses.EvidentialRegression})
    else:
       pre_trained_model = tf.keras.models.load_model(source)
    pre_model_config = pre_trained_model.get_config()
    pre_model_config["layers"][0]["config"]["batch_input_shape"] = (None, 104, 104, new_channels)

    # Load the post-training model:
    if evidential:
        post_model = tf.keras.Model.from_config(pre_model_config, custom_objects={'Conv2DNormalGamma': edl.layers.Conv2DNormalGamma, 'EvidentialRegression':edl.losses.EvidentialRegression})
    else:
        post_model = tf.keras.Model.from_config(pre_model_config)
    
    post_model_config = post_model.get_config()
    post_model_layers = [post_model_config['layers'][x]['name'] for x in range(len(post_model_config['layers']))]
    conv_layer = post_model_layers[1]

    # Iterate across & copy weights:
    for layer in pre_trained_model.layers:
        if layer.name in post_model_layers:
            if (layer.get_weights() != []): 
                target_layer = post_model.get_layer(layer.name)
                if layer.name == conv_layer:
                    weights = layer.get_weights()[0]
                    biases = layer.get_weights()[1]                    
                    weights_extra_channels = copy_weights(weights, new_channels - old_channel_count)
                    new_weights = np.concatenate((weights, weights_extra_channels), axis=-2)
                    target_layer.set_weights([new_weights, biases])                    
                    target_layer.trainable == True
                else:
                    target_layer.set_weights(layer.get_weights())
                    target_layer.trainable=True
    
    del pre_trained_model

    if evidential:
        post_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=edl.losses.EvidentialRegression)
    else:
        post_model.compile(optimizer=Adam(), loss="mean_squared_error", metrics="accuracy")
    return post_model, data

In [1]:
def post_train(EXPERIMENT_ID, pre_path, destination, evidential, EPOCHS):
    """
    Sets-up the model and data loader for POST-Training (2011-2018)
    Proceeds to train the network, saving incremental improvements (based on the loss).
    """
    post_model, data = create_transfer_network(pre_path, evidential)
    es = EarlyStopping(monitor='loss', verbose=1, min_delta=0.000001, patience=3)
    mc = ModelCheckpoint(destination, monitor='loss', mode='min', verbose=1, save_best_only=True)
    post_model.fit(data, epochs=EPOCHS, verbose=1, callbacks = [es,mc])

In [ ]:
def baseline_train(EXPERIMENT_ID, pre_path, destination, evidential, EPOCHS):
    """
    Sets-up the model and data loader for post training using ERA-5 and SIC data only (2011-2018). Consequently, the number of channels remains the same.
    """
    data, channels = load_data_and_channels(f"./configs/{EXPERIMENT_ID}/post_training_baseline.json")
        
    if evidential:
        baseline_model = tf.keras.models.load_model(pre_path, custom_objects={'Conv2DNormalGamma': edl.layers.Conv2DNormalGamma, 'EvidentialRegression':edl.losses.EvidentialRegression})
        baseline_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=edl.losses.EvidentialRegression)
    else:
        baseline_model = tf.keras.models.load_model(pre_path)
    
    es = EarlyStopping(monitor='loss', verbose=1, patience=5)
    mc = ModelCheckpoint(destination, monitor='loss', mode='min', verbose=1, save_best_only=True)
    baseline_model.fit(data, epochs=EPOCHS, verbose=1, callbacks = [es,mc])

## **Experiments**

In [ ]:
# EXPERIMENT CONFIGURATION
EXPERIMENT_ID = 401
EPOCHS = 15
EVIDENTIAL = True
experiment_path = os.path.join(cfg.MODELS_FOLDER, 'models', str(EXPERIMENT_ID))

# SETUP EXPERIMENT FILE STRUCTURE
if not os.path.isfile(experiment_path):
    os.makedirs(experiment_path)

# ================= PRE-TRAINING =================
#Setup & train the pre-trained network.
pre_train_path = os.path.join(experiment_path, f'pre_training_{EXPERIMENT_ID}.h5')
pre_train(EXPERIMENT_ID, experiment_path, pre_train_path, EVIDENTIAL, EPOCHS)

### **Post-Training, "Data-Rich" Model**

In [ ]:
EXPERIMENT_ID = 401
EPOCHS = 15

# ================= POST-TRAINING, ADDITIONAL VARIABLES =================
experiment_path = os.path.join(cfg.MODELS_FOLDER, 'models', str(EXPERIMENT_ID))
pre_train_path = os.path.join(experiment_path, f'pre_training_{EXPERIMENT_ID}.h5')
post_train_path = os.path.join(experiment_path, f'post_training_{EXPERIMENT_ID}.h5')
post_train(EXPERIMENT_ID, pre_train_path, post_train_path, True, EPOCHS)

### **Post-Training, Baseline Model**

In [ ]:
EXPERIMENT_ID = 401
EPOCHS = 15

# ================= POST-TRAINING, NO ADDITIONAL VARIABLES =================
experiment_path = os.path.join(cfg.MODELS_FOLDER, 'models', str(EXPERIMENT_ID))
pre_trained_path = os.path.join(experiment_path, f'pre_training_{EXPERIMENT_ID}.h5')
baseline_path = os.path.join(experiment_path, f'post_baseline_{EXPERIMENT_ID}.h5')
baseline_train(EXPERIMENT_ID, pre_trained_path, baseline_path, True, EPOCHS)

## **MODEL EVALUATION**

In [ ]:
def load_model(source):
    # COMMENT OUT BASED ON IF THE EVIDENTIAL OR TRADITIONAL MODEL IS USED!
    model = tf.keras.models.load_model(source, custom_objects={'Conv2DNormalGamma': edl.layers.Conv2DNormalGamma, 'EvidentialRegression':edl.losses.EvidentialRegression})
    #model = tf.keras.models.load_model(source)
    return model

In [ ]:
def reshape_array(sample):
    sample = np.moveaxis(sample, 2,3)
    sample = np.moveaxis(sample, 1,2)
    return sample

In [ ]:
def generate_non_evidenced_predictions(data_loader, model, save_location):    
    # Save the results (deleting previous results):    
    if not os.path.exists(save_location):
        os.makedirs(save_location)
    
    X,Y = data_loader.__getitem__(0)
    print(data_loader.dates)
    outputs = model.predict(X)

    # ========== MODEL ESTIMATES (MU) ==========
    estimates = reshape_array(outputs)
    estimates_loc = os.path.join(save_location, "non_evidenced_estimates.npy")
    np.save(estimates_loc, estimates)
    return

### Evidential Regression:
- The network learns a number of parameters during evidential regression which are subsequently used to determine error, mean guess etc.
- https://github.com/aamini/evidential-deep-learning

`import evidential_deep_learning as edl`

In [ ]:
def generate_predictions(data_loader, model, save_location):
    X,Y = data_loader.__getitem__(0)
    outputs = model.predict(X)
    mu, v, alpha, beta = tf.split(outputs, 4, axis=-1)
    
    # ========== MODEL ESTIMATES (MU) ==========
    estimates = reshape_array(np.clip(mu, 0, 1))
    estimates_loc = os.path.join(save_location, "estimates.npy")

    # ========== EPISTEMIC UNCERTAINTY (VARIANCE OF MU) ==========
    epistemic = reshape_array(tf.sqrt(beta/(v*(alpha-1))))
    epistemic_loc = os.path.join(save_location, "epistemic.npy")

    # ========== ALEATORIC UNCERTAINTY (...) ==========
    aleatoric = reshape_array(tf.sqrt(beta/(alpha-1)))
    aleatoric_loc = os.path.join(save_location, "aleatoric.npy")

    # ============ VARIANCE ==================
    variance_loc = os.path.join(save_location, "variance.npy")

    # ========== GROUND TRUTH (SIC [Y]) ==========
    Y = reshape_array(Y)
    Y_loc = os.path.join(save_location, "truth.npy")

    # Save the results (deleting previous results):    
    if not os.path.exists(save_location):
        os.makedirs(save_location)

    np.save(estimates_loc, estimates)
    np.save(epistemic_loc, epistemic)
    np.save(aleatoric_loc, aleatoric)
    np.save(variance_loc, v)
    np.save(Y_loc, Y)
    return outputs, Y

In [3]:
def load_and_predict(type, test_config_name):
    """
    Loads a specific model & generates predictions for a given input by making a Data Loader from a specified configuration file
    """
    path = os.path.join(experiment_path, f'{type}_{EXPERIMENT_ID}.h5')
    model = load_model(path)
    test_data, new_channels = load_data_and_channels(f"./configs/{EXPERIMENT_ID}/{test_config_name}.json")
    del new_channels
    save_loc = os.path.join(experiment_path, type)
    return generate_predictions(test_data, model, save_loc)

### **Generate Output**

In [ ]:
EXPERIMENT_ID = 401
experiment_path = os.path.join(cfg.MODELS_FOLDER, 'models', str(EXPERIMENT_ID))
load_and_predict('post_baseline', 'baseline_testing')
load_and_predict('post_training', 'testing')

## **Variable Importance Analysis**
Determines the error induced by assigning 0 to variables in turn (both in regards to extent and RMSE scores).

In [ ]:
def create_heatmap_dict(EXPERIMENT_ID, type):
    
    cfg = open(f"./configs/{EXPERIMENT_ID}/testing.json")
    cfg=json.load(cfg)
    variables = ["sic", "2m_temperature", "sea_surface_temperature", "mean_sea_level_pressure", "surface_solar_radiation_upwards", "surface_net_solar_radiation", "10m_v_component_of_wind", "sit", "u_drift", "v_drift"]
    path = os.path.join(f"./models/{EXPERIMENT_ID}/post_training_{EXPERIMENT_ID}.h5")
    model = tf.keras.models.load_model(path)

    new_metrics = []
    for var in variables:
        # Setup the new model and the dataloader:
        cfg["permute_var"] = var
        data_object = data_loader_perm.ThesisDataLoader(cfg)
        X,Y = data_object.__getitem__(0)
        outputs = model.predict(X)
        estimates = reshape_array(outputs)
        np.save(f"./models/{EXPERIMENT_ID}/var_importance{var}.npy", estimates)
        
        if type == "extent":
            new_metrics.append(spatial_extent(estimates))
        else:
            new_metrics.append(generate_RMSE(estimates))

    # Get the original metrics:
    cfg["permute_var"] = None
    data_object = data_loader_perm.ThesisDataLoader(cfg)
    X,Y = data_object.__getitem__(0)
    outputs = model.predict(X)
    estimates = reshape_array(outputs)

    if type == "extent":
        original_metrics = spatial_extent(estimates)
    else:
        original_metrics = generate_RMSE(estimates)

    variables = ["SIC", "2m Temperature", "SST", "Mean Sea Level Pressure", "Surface Solar Radiation (Down)", "Surface Solar Radiation (Net)", "10m Wind (v)", "SIT", "Drift (u)", "Drift (v)"]
    var_importance = {}

    count=0
    for key in variables:
        if type == "extent": 
            var_importance[key] = original_metrics[count]-new_metrics[count]
        else:
            var_importance[key] = new_metrics[count]/original_metrics[count]
        count+=1

    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    var_imp = pd.DataFrame(var_importance)
    var_imp = var_imp.rename({i:months[i] for i in range(0,12)})
    return var_imp.transpose()

In [ ]:
f,(ax1,ax2,ax3, axcb) = plt.subplots(1,4,  gridspec_kw={'width_ratios':[2.5,2.5,2.5,0.2]}, figsize=(12,4))
ax1.get_shared_y_axes().join(ax2,ax3)

g1 = sns.heatmap(month_1_extent,cmap="RdBu",cbar=False,ax=ax1)
g1.set_ylabel('')
g1.set_xlabel('')
g1.set_title("1 Month")
g2 = sns.heatmap(month_2_extent,cmap="RdBu",cbar=False,ax=ax2)
g2.set_ylabel('')
g2.set_xlabel('')
g2.set_yticks([])
g2.set_title("2 Month")
g3 = sns.heatmap(month_3_extent,cmap="RdBu",ax=ax3, cbar_ax=axcb)
g3.set_ylabel('')
g3.set_xlabel('')
g3.set_title("3 Month")
g3.set_yticks([])

plt.suptitle("Variable Sensitivity for Sea Ice Extent")


plt.show()

In [ ]:
import scipy
NSIDC = load_year(YEAR)
mask = NSIDC[0]
    
def spatial_extent(estimates):
    thickness = []
    nsidc = []
    ground_truth = np.load("../data/processed_data/sic/2010_01.npy")
        
    for i in range(0, 12):
        mean_thick = pre_process_variable(estimates[i][0], ground_truth)
        mask = np.zeros((100, 100))
        mask[np.where(mean_thick>0.15)] = 1
        thickness.append(np.sum(mask)*80/10000)
    return thickness


In [ ]:
maximums = ["2012_01.npy", "2013_02.npy", "2013_03.npy", "2010_04.npy", "2010_05.npy", "2013_06.npy", "2013_07.npy", "2014_08.npy", "2014_09.npy", "2013_10.npy", "2014_11.npy", "2014_12.npy"]
sic_path = "../data/datasets/post_standard/sic/"
def mask_maximum(sample, ground_truth, maximum):
    land = np.where(np.isnan(ground_truth))
    sample[land]= 0
    maximum_sic = np.load(sic_path+maximum)
    mask = np.zeros((104, 104))
    mask[np.where(maximum_sic>0.15)]=1
    sample = sample*mask
    return sample[:100, :100]*100

def generate_RMSE(estimates):
    ground_truth = np.load("../data/processed_data/sic/2010_01.npy")
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    RMSE = []
    
    for m in range(0, 12):
        print(months[m])
        ice_sample = mask_maximum(estimates[0][0], ground_truth, maximums[m])
        if m <9:
            month = f'0{m+1}'
        else:
            month = f'{m+1}'
        true_ice = mask_maximum(np.load(f'../data/datasets/post_standard/sic/2019_{month}.npy'), ground_truth, maximums[m])
        RMSE.append(mean_squared_error(true_ice, ice_sample, squared = False))
    return RMSE

## **GENERAL ERROR METRICS**

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np

# =========== CHANGE ===========
EXPERIMENT_ID = 302
# ===============================

maximums = ["2012_01.npy", "2013_02.npy", "2013_03.npy", "2010_04.npy", "2010_05.npy", "2013_06.npy", "2013_07.npy", "2014_08.npy", "2014_09.npy", "2013_10.npy", "2014_11.npy", "2014_12.npy"]
sic_path = "../data/datasets/post_standard/sic/"

def mask_maximum(sample, ground_truth, maximum):
    land = np.where(np.isnan(ground_truth))
    sample[land]= 0
    maximum_sic = np.load(sic_path+maximum)
    mask = np.zeros((104, 104))
    mask[np.where(maximum_sic>0.15)]=1
    sample = sample*mask
    return sample[:100, :100]*100

def generate_RMSE(estimates):
    ground_truth = np.load("../data/processed_data/sic/2010_01.npy")
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    RMSE = []
    
    for m in range(0, 12):
        ice_sample = mask_maximum(estimates[0][0], ground_truth, maximums[m])
        if m <9:
            month = f'0{m+1}'
        else:
            month = f'{m+1}'
        true_ice = mask_maximum(np.load(f'../data/datasets/post_standard/sic/2019_{month}.npy'), ground_truth, maximums[m])
        RMSE.append(mean_squared_error(true_ice, ice_sample, squared = False))
    return RMSE

def generate_error_metrics(estimates):
    ground_truth = np.load("../data/processed_data/sic/2010_01.npy")
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    RMSE = []
    NRMSE = []
    NSE = []
    MAE = []

    for m in range(0, 12):
        ice_sample = mask_maximum(estimates[0][0], ground_truth, maximums[m])
        if m <9:
            month = f'0{m+1}'
        else:
            month = f'{m+1}'

        true_ice = mask_maximum(np.load(f'../data/datasets/post_standard/sic/2019_{month}.npy'), ground_truth, maximums[m])

         # CALCULATE RMSE:
        RMSE.append(mean_squared_error(true_ice, ice_sample, squared = False))

        # CALCULATE nRMSE:
        NRMSE.append(RMSE[m-1]/np.std(true_ice))

        # CALCULATE MAE:
        MAE.append(mean_absolute_error(true_ice, ice_sample))

        # CALCULATE NSE:
        NSE.append((1-(np.sum((true_ice-ice_sample)**2)/np.sum((true_ice-np.mean(true_ice))**2))))
    
    print(RMSE)
    print("=================================================================================================================")
    print(f'|\tMonth\t|\tRMSE\t\t|\tnRMSE\t\t|\tMAE\t\t|\tNSE\t\t|')
    print("=================================================================================================================")
    for month_count, month in enumerate(months):
        print(f'|\t{month}\t|\t{RMSE[month_count-1]:.3f}\t\t|\t{NRMSE[month_count-1]:.3f}\t\t|\t{MAE[month_count-1]:.3f}\t\t|\t{NSE[month_count-1]:.3f}\t\t|')
    print("=================================================================================================================")
    # APPLY JULIANs METRICS?

    #print(np.mean(np.array(MAE)))

    return 

thickness_predictions = np.load(f"./models/{EXPERIMENT_ID}/post_training/non_evidenced_estimates.npy")
baseline_predictions = np.load(f"./models/{EXPERIMENT_ID}/post_baseline/non_evidenced_estimates.npy")
print("BASELINE")
generate_error_metrics(baseline_predictions)
print("THICKNESS")
generate_error_metrics(thickness_predictions)


## **Plot Results** 
### **Requires Kernel Restart**

In [ ]:
import json
import numpy as np
import os
import sys
sys.path.append("../utilities")
import U_config as cfg
import U_plots as p

# MODIFY THESE
YEAR = 2019
EXPERIMENT_ID =401
TYPE = "non_evidenced_estimates"
MONTHS = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
DATES = [f"{month} {YEAR}" for month in MONTHS]

In [ ]:
def load_year(year):
    truth = []
    for i in range(1, 13):
        truth.append(np.load(f"../data/processed_data/sic/{year}_{f'0{i}' if i<10 else i}.npy"))
    return np.array(truth)

In [ ]:
def pre_process_variable(sample, ground_truth):
    sample[sample>1] = 1
    sample[sample<0] = 0
    land = np.where(np.isnan(ground_truth))
    sample[land]= np.nan
    return sample[:100, :100]



In [ ]:
def pre_process_thick(sample, ground_truth):
    land = np.where(np.isnan(ground_truth))
    sample[land]= np.nan
    return sample[:100, :100]

In [ ]:
maximums = ["2012_01.npy", "2013_02.npy", "2013_03.npy", "2010_04.npy", "2010_05.npy", "2013_06.npy", "2013_07.npy", "2014_08.npy", "2014_09.npy", "2013_10.npy", "2014_11.npy", "2014_12.npy"]
sic_path = "../data/datasets/post_standard/sic/"

def mask_non_active_region(sample, ground_truth, maximum):
    land = np.where(np.isnan(ground_truth))
    sample[land]= np.nan
    maximum_sic = np.load(sic_path+maximum)
    sample[np.where(maximum_sic<0.15)]=np.nan
    return sample[:100, :100]

    

#### **Main Method for Results**

In [ ]:
# Non-Evidenced Results
plot_object = p.plots()
NSIDC = load_year(YEAR)
mask = NSIDC[0]
EXPERIMENT_ID = 401

LAG_MONTHS = ["November", "December", "January", "February", "March", "April", "May", "June", "July", "August", "September", "October"]

#thickness_predictions = np.load(f"./models/{EXPERIMENT_ID}/post_training/non_evidenced_estimates.npy")
#baseline_predictions = np.load(f"./models/{EXPERIMENT_ID}/post_baseline/non_evidenced_estimates.npy")
#thickness_predictions = np.load(f"./models/{EXPERIMENT_ID}/post_training/estimates.npy")
#baseline_predictions = np.load(f"./models/{EXPERIMENT_ID}/post_baseline/estimates.npy")

thickness_epi = np.load(f"./models/{EXPERIMENT_ID}/post_training/epistemic.npy")
baseline_epi = np.load(f"./models/{EXPERIMENT_ID}/post_baseline/epistemic.npy")

#thickness_aleo = np.load(f"./models/{EXPERIMENT_ID}/post_training/aleatoric.npy")
#baseline_aleo = np.load(f"./models/{EXPERIMENT_ID}/post_baseline/aleatoric.npy")
# thickness_plots = f"./models/{EXPERIMENT_ID}/post_training/"
# baseline_plots = f"./models/{EXPERIMENT_ID}/post_baseline/"
# comparitive_plots = f"./models/{EXPERIMENT_ID}/comparative"

# plotting uncertainty;
for i in range(4, 12):
    thick = mask_non_active_region(thickness_epi[i][0], mask, maximums[i])
    base = mask_non_active_region(baseline_epi[i][0], mask, maximums[i])
    plot_object.plot_uncertainty(NSIDC[i], thick, base, DATES[i], comparitive_plots+f"/{DATES[i]}_epistemic.png", 0, 3, "plasma", "Epistemic Uncertainty of the Observational Model", "Epistemic Uncertainty of the Baseline Model", contour=False)

    #a_thick = mask_non_active_region(thickness_aleo[i][0], mask, maximums[i])
    #a_base = mask_non_active_region(baseline_aleo[i][0], mask, maximums[i])
    #plot_object.plot_uncertainty(NSIDC[i], a_thick, a_base, DATES[i], comparitive_plots+f"/{DATES[i]}_aleatoric.png", 0, 3, "plasma", "Aleatoric Uncertainty of the Observational Model", "Aleatoric Uncertainty of the Baseline Model", contour=False)
    
    #thick = pre_process_variable(thickness_predictions[i][0], mask)
    #base = pre_process_variable(baseline_predictions[i][0], mask)
    #plot_object.plot_prediction(NSIDC[i]*100, thick*100, base*100, DATES[i], comparitive_plots+f"/{DATES[i]}_estimates.png", contour=False)
    
    #plot_object.plot_anom(NSIDC[i]*100, thick*100, base*100, DATES[i], comparitive_plots+f"/{DATES[i]}_anomalies.png", col='#5c5c5c', contour=False)
    

In [ ]:
# Non-Evidenced Results
plot_object = p.plots()

NSIDC = load_year(YEAR)
mask = NSIDC[0]
EXPERIMENT_ID = 303
LAG_MONTHS = ["November", "December", "January", "February", "March", "April", "May", "June", "July", "August", "September", "October"]

thickness_predictions = np.load(f"./models/{EXPERIMENT_ID}/post_training/non_evidenced_estimates.npy")
baseline_predictions = np.load(f"./models/{EXPERIMENT_ID}/post_baseline/non_evidenced_estimates.npy")
#thickness_data = [np.load(f'../data/processed_data/sit/2018_12.npy'), np.load(f'../data/processed_data/sit/2019_01.npy'), np.load(f'../data/processed_data/sit/2019_02.npy'), np.load(f'../data/processed_data/sit/2019_03.npy')
#                  , np.load(f'../data/processed_data/sit/2019_04.npy'), np.load(f'../data/processed_data/sit/2019_05.npy'), np.load(f'../data/processed_data/sit/2019_06.npy'), np.load(f'../data/processed_data/sit/2019_07.npy')
#                  , np.load(f'../data/processed_data/sit/2019_08.npy'), np.load(f'../data/processed_data/sit/2019_09.npy'), np.load(f'../data/processed_data/sit/2019_10.npy'), np.load(f'../data/processed_data/sit/2019_11.npy')]

thickness_plots = f"./models/{EXPERIMENT_ID}/post_training/"
baseline_plots = f"./models/{EXPERIMENT_ID}/post_baseline/"
comparitive_plots = f"./models/{EXPERIMENT_ID}/comparative"

for i in range(0, 12):
    thick = pre_process_variable(thickness_predictions[i][0], mask)
    base = pre_process_variable(baseline_predictions[i][0], mask)
    #thickness = pre_process_thick(thickness_data[i], mask)
    plot_object.plot_prediction(NSIDC[i]*100, thick*100, base*100, DATES[i], comparitive_plots+f"/{DATES[i]}_estimates.png", contour=False)
    plot_object.plot_anom(NSIDC[i]*100, thick*100, base*100, DATES[i], comparitive_plots+f"/{DATES[i]}_anomalies.png", col='#5c5c5c', contour=False)
    #plot_object.plot_anom_thick(NSIDC[i]*100, thick*100, base*100, thickness, LAG_MONTHS[i], DATES[i], 0, 5, 1, comparitive_plots+f"/{DATES[i]}_anomalies_thick.png", col='#5c5c5c', contour=False)
    
    

In [ ]:
import scipy

NSIDC = load_year(YEAR)
mask = NSIDC[0]

def pre_process_variable(sample, ground_truth):
    land = np.where(np.isnan(ground_truth))
    sample[land]= 0
    sample[sample>1] = 1
    sample[sample<0] = 0
    return sample[:100, :100]

def spatial_extent(EXPERIMENT_ID):
    
    thickness_mu = np.load(f"./models/{EXPERIMENT_ID}/post_training/non_evidenced_estimates.npy")
    baseline_mu = np.load(f"./models/{EXPERIMENT_ID}/post_baseline/non_evidenced_estimates.npy")

    sic_data = [np.load(f'../data/processed_data/sic/2019_01.npy'), np.load(f'../data/processed_data/sic/2019_02.npy'), np.load(f'../data/processed_data/sic/2019_03.npy')
                  , np.load(f'../data/processed_data/sic/2019_04.npy'), np.load(f'../data/processed_data/sic/2019_05.npy'), np.load(f'../data/processed_data/sic/2019_06.npy'), np.load(f'../data/processed_data/sic/2019_07.npy')
                  , np.load(f'../data/processed_data/sic/2019_08.npy'), np.load(f'../data/processed_data/sic/2019_09.npy'), np.load(f'../data/processed_data/sic/2019_10.npy'), np.load(f'../data/processed_data/sic/2019_11.npy'), np.load(f'../data/processed_data/sic/2019_12.npy')]

    ground_truth = np.load("../data/processed_data/sic/2010_01.npy")

    obs_res = []
    base_res = []  
    nsidc = []

    for i in range(0, 12):

        thickness = pre_process_variable(thickness_mu[i][0], ground_truth)
        mask = np.zeros((100, 100))
        mask[np.where(thickness>0.15)] = 1
        obs_res.append(np.sum(mask)*80/10000)
        
        baseline = pre_process_variable(baseline_mu[i][0], ground_truth)
        mask = np.zeros((100, 100))
        mask[np.where(baseline>0.15)] = 1
        base_res.append(np.sum(mask)*80/10000)
        
        sic = sic_data[i]
        mask = np.zeros((100, 100))
        mask[np.where(sic>0.15)] = 1
        nsidc.append(np.sum(mask)*80/10000)

    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    print("============================================================================================================================")
    print("|\tMonth\t|\tNSIDC\t|\tBaseline Extent\t|\tDelta\t|\tThickness Extent\t|\tDelta\t|")
    print("============================================================================================================================")
    
    for i in range(0, 12):
        print(f'|\t{months[i]}\t|\t{nsidc[i]}\t|\t{base_res[i]}\t\t|\t{nsidc[i]-base_res[i]:.3f}\t|\t{obs_res[i]}\t\t\t|\t{nsidc[i]-obs_res[i]:.3f}\t|')
    print("============================================================================================================================")
    
    return base_res, obs_res, nsidc

baseline, thickness, nsidc = spatial_extent(302)

In [ ]:
print(bl)

In [ ]:
import matplotlib.pyplot as plt
from scipy import interpolate
import numpy as np


months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
fig, (ax1) = plt.subplots(1,1, figsize=(12, 4), constrained_layout=True)

ax1.plot(months, baseline,  label='Baseline Extent', color='#1cb0ff')
ax1.plot(months, thickness,  label='Thickness Extent', color='#ff491c')
ax1.plot(months, nsidc,  label='NSIDC Extent', color='gray', linestyle='--')

plt.suptitle(f"Comparison of SIE (Millions 10^3 km^3)")
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from scipy import interpolate
import numpy as np

plt.style.use('seaborn-whitegrid')
f = plt.figure(figsize=[10,4])
ax = plt.axes()
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December", ]

# Lags : 
#january_b = baseline[0]
#baseline.pop(0)
#baseline.append(january_b)
#january_b = thickness[0]
#thickness.pop(0)
#thickness.append(january_b)

plt.plot(months, baseline,  label='Baseline Model', color='#1cb0ff')
plt.plot(months, thickness, label='Observational Model', color='#ff491c')
plt.plot(months, nsidc, linestyle='dashed', label='NSIDC', color='#595959')
plt.axis('tight')
plt.title("Monthly Averaged Sea-Ice Extent for 1-Month Forecasts (Millions Km^2)\nLAGGED FORECAST")
plt.xlabel("Months")
plt.ylabel("Extent in Millions Km^2")
legend=plt.legend()
plt.show()

In [ ]:
import math
import matplotlib.pyplot as plt

maximums = ["2012_01.npy", "2013_02.npy", "2013_03.npy", "2010_04.npy", "2010_05.npy", "2013_06.npy", "2013_07.npy", "2014_08.npy", "2014_09.npy", "2013_10.npy", "2014_11.npy", "2014_12.npy"]
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December", ]
sic_path = "../data/datasets/post_standard/sic/"

def mask_maximum(sample, ground_truth, maximum):
    land = np.where(np.isnan(ground_truth))
    sample[land]= 0
    maximum_sic = np.load(sic_path+maximum)
    mask = np.zeros((104, 104))
    mask[np.where(maximum_sic>0.15)]=1
    sample = sample*mask
    return sample[:100, :100]*100


def SIE_histogram(experiment_ID, path):
    thickness_predictions = np.load(f"./models/{experiment_ID}/post_training/non_evidenced_estimates.npy")
    baseline_predictions = np.load(f"./models/{experiment_ID}/post_baseline/non_evidenced_estimates.npy")
    sic_data = [np.load(f'../data/processed_data/sic/2019_01.npy'), np.load(f'../data/processed_data/sic/2019_02.npy'), np.load(f'../data/processed_data/sic/2019_03.npy')
                  , np.load(f'../data/processed_data/sic/2019_04.npy'), np.load(f'../data/processed_data/sic/2019_05.npy'), np.load(f'../data/processed_data/sic/2019_06.npy'), np.load(f'../data/processed_data/sic/2019_07.npy')
                  , np.load(f'../data/processed_data/sic/2019_08.npy'), np.load(f'../data/processed_data/sic/2019_09.npy'), np.load(f'../data/processed_data/sic/2019_10.npy'), np.load(f'../data/processed_data/sic/2019_11.npy'), np.load(f'../data/processed_data/sic/2019_12.npy')]

    ground_truth = np.load("../data/processed_data/sic/2010_01.npy")        
    
    for i in range(0, 12):
        f, (ax1, ax2) = plt.subplots(1,2, figsize=(12, 3))
        month = f'0{i+1}' if i <9 else f'{i+1}'

        sic = mask_maximum(np.load(f'../data/datasets/post_standard/sic/2019_{month}.npy'), ground_truth, maximums[i])
        ax2.hist(sic.ravel(), bins=np.arange(0, 101, 2),  color='#58595c', log=True, edgecolor = "black", label="NSIDC")
        ax2.set_title("Baseline Model")
        
        base = mask_maximum(baseline_predictions[i][0], ground_truth, maximums[i])
        ax2.hist(base.ravel(), bins=np.arange(0, 101, 2),  color='#3882c2', log=True, edgecolor = "#0d416e", alpha=0.5, label="Baseline")
        ax2.legend()
        
        ax1.hist(sic.ravel(), bins=np.arange(0, 101, 2),  color='#58595c', log=True, edgecolor = "black", label="NSIDC")
        ax1.set_title("Observational Model")
        thick = mask_maximum(thickness_predictions[i][0], ground_truth, maximums[i])
        ax1.hist(thick.ravel(), bins=np.arange(0, 101, 2),  color='#f59042', log=True, edgecolor = "#c2590a", alpha=0.5, label="Observational")
        ax1.legend()

        plt.suptitle(f"Comparison of SIC Distribution relative to NSIDC between Models for {months[i]}")
        plt.tight_layout()
        plt.savefig(path+f"_{months[i]}.png")
        plt.show()

#https://stackoverflow.com/questions/64156602/how-to-build-a-histogram-of-numpy-2-dimensional-array
#https://stackoverflow.com/questions/64156602/how-to-build-a-histogram-of-numpy-2-dimensional-array

SIE_histogram(303, "./models/303/")  

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np

# =========== CHANGE ===========
EXPERIMENT_ID = 301
# ===============================

maximums = ["2012_01.npy", "2013_02.npy", "2013_03.npy", "2010_04.npy", "2010_05.npy", "2013_06.npy", "2013_07.npy", "2014_08.npy", "2014_09.npy", "2013_10.npy", "2014_11.npy", "2014_12.npy"]
sic_path = "../data/datasets/post_standard/sic/"

def mask_maximum(sample, ground_truth, maximum):
    land = np.where(np.isnan(ground_truth))
    sample[land]= 0
    maximum_sic = np.load(sic_path+maximum)
    mask = np.zeros((104, 104))
    mask[np.where(maximum_sic>0.15)]=1
    sample = sample*mask
    return sample[:100, :100]*100

def generate_error_metrics(estimates):
    ground_truth = np.load("../data/processed_data/sic/2010_01.npy")
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    RMSE = []
    NRMSE = []
    NSE = []
    MAE = []

    for m in range(0, 12):
        ice_sample = mask_maximum(estimates[0][0], ground_truth, maximums[m])
        if m <9:
            month = f'0{m+1}'
        else:
            month = f'{m+1}'
        true_ice = mask_maximum(np.load(f'../data/datasets/post_standard/sic/2019_{month}.npy'), ground_truth, maximums[m])

         # CALCULATE RMSE:
        RMSE.append(mean_squared_error(true_ice, ice_sample, squared = False))

        # CALCULATE nRMSE:
        NRMSE.append(RMSE[m-1]/np.std(true_ice))

        # CALCULATE MAE:
        MAE.append(mean_absolute_error(true_ice, ice_sample))

        # CALCULATE NSE:
        NSE.append((1-(np.sum((true_ice-ice_sample)**2)/np.sum((true_ice-np.mean(true_ice))**2))))
    
    print("=================================================================================================================")
    print(f'|\tMonth\t|\tRMSE\t\t|\tnRMSE\t\t|\tMAE\t\t|\tNSE\t\t|')
    print("=================================================================================================================")
    for month_count, month in enumerate(months):
        print(f'|\t{month}\t|\t{RMSE[month_count-1]:.3f}\t\t|\t{NRMSE[month_count-1]:.3f}\t\t|\t{MAE[month_count-1]:.3f}\t\t|\t{NSE[month_count-1]:.3f}\t\t|')
    print("=================================================================================================================")
    # APPLY JULIANs METRICS?

    print(np.mean(np.array(MAE)))

    return

thickness_predictions = np.load(f"./models/{EXPERIMENT_ID}/post_training/non_evidenced_estimates.npy")
baseline_predictions = np.load(f"./models/{EXPERIMENT_ID}/post_baseline/non_evidenced_estimates.npy")
print("BASELINE")
generate_error_metrics(baseline_predictions)
print("THICKNESS")
generate_error_metrics(thickness_predictions)

